In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import model_selection # Library untuk proses prediksi
from sklearn.model_selection import KFold, RepeatedKFold # Library untuk penerapan KFold Validation
from sklearn.model_selection import cross_val_score # Library untuk menampilkan nilai pengujian Cross Validation (nilai return nya berupa array)
from sklearn.model_selection import cross_validate # ...
from sklearn.model_selection import train_test_split # Library untuk memisahkan data menjadi data training:testing
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # library untuk menerapkan gridsearch
from sklearn.metrics import roc_auc_score , roc_curve , make_scorer , accuracy_score , precision_score , recall_score , f1_score, confusion_matrix, classification_report # Menampilkan hasil tes

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [63]:
food = pd.read_csv('data/indonesia-food-prices-1.csv')
food

,date,cmname,unit,category,price,currency,country,admname,adm1id,mktname,mktid,cmid,ptid,umid,catid,sn,default
0,#date,#item+name,#item+unit,#item+type,#value,#currency,#country+name,#adm1+name,#adm1+code,#name+market,NaN,#item+code,NaN,NaN,#item+type+code,#meta+id,NaN
1,2007-01-15,Rice - Retail,KG,cereals and tubers,5941.9752,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
2,2007-02-15,Rice - Retail,KG,cereals and tubers,6445.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
3,2007-03-15,Rice - Retail,KG,cereals and tubers,6414.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
4,2007-04-15,Rice - Retail,KG,cereals and tubers,6083.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,2013-08-15,Fuel (kerosene) - Retail,L,non-food,6233.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1390,2013-09-15,Fuel (kerosene) - Retail,L,non-food,6244.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1391,2013-10-15,Fuel (kerosene) - Retail,L,non-food,6243.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1392,2013-11-15,Fuel (kerosene) - Retail,L,non-food,6296.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN


Preparing dataset

In [64]:
food1 = food.copy()
food1

,date,cmname,unit,category,price,currency,country,admname,adm1id,mktname,mktid,cmid,ptid,umid,catid,sn,default
0,#date,#item+name,#item+unit,#item+type,#value,#currency,#country+name,#adm1+name,#adm1+code,#name+market,NaN,#item+code,NaN,NaN,#item+type+code,#meta+id,NaN
1,2007-01-15,Rice - Retail,KG,cereals and tubers,5941.9752,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
2,2007-02-15,Rice - Retail,KG,cereals and tubers,6445.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
3,2007-03-15,Rice - Retail,KG,cereals and tubers,6414.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
4,2007-04-15,Rice - Retail,KG,cereals and tubers,6083.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,2013-08-15,Fuel (kerosene) - Retail,L,non-food,6233.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1390,2013-09-15,Fuel (kerosene) - Retail,L,non-food,6244.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1391,2013-10-15,Fuel (kerosene) - Retail,L,non-food,6243.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1392,2013-11-15,Fuel (kerosene) - Retail,L,non-food,6296.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN


In [65]:
# Menghapus data row pertama
food1 = food1.tail(food1.shape[0] -1)
food1

,date,cmname,unit,category,price,currency,country,admname,adm1id,mktname,mktid,cmid,ptid,umid,catid,sn,default
1,2007-01-15,Rice - Retail,KG,cereals and tubers,5941.9752,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
2,2007-02-15,Rice - Retail,KG,cereals and tubers,6445.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
3,2007-03-15,Rice - Retail,KG,cereals and tubers,6414.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
4,2007-04-15,Rice - Retail,KG,cereals and tubers,6083.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
5,2007-05-15,Rice - Retail,KG,cereals and tubers,5955.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,1,974_52_15_5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,2013-08-15,Fuel (kerosene) - Retail,L,non-food,6233.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1390,2013-09-15,Fuel (kerosene) - Retail,L,non-food,6244.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1391,2013-10-15,Fuel (kerosene) - Retail,L,non-food,6243.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN
1392,2013-11-15,Fuel (kerosene) - Retail,L,non-food,6296.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,8,974_283_15_15,NaN


In [66]:
food1.drop(columns=['default', 'sn', 'mktname', 'currency', 'country', 'admname', 'adm1id'])

,date,cmname,unit,category,price,mktid,cmid,ptid,umid,catid
1,2007-01-15,Rice - Retail,KG,cereals and tubers,5941.9752,974.0,52,15.0,5.0,1
2,2007-02-15,Rice - Retail,KG,cereals and tubers,6445.0,974.0,52,15.0,5.0,1
3,2007-03-15,Rice - Retail,KG,cereals and tubers,6414.0,974.0,52,15.0,5.0,1
4,2007-04-15,Rice - Retail,KG,cereals and tubers,6083.0,974.0,52,15.0,5.0,1
5,2007-05-15,Rice - Retail,KG,cereals and tubers,5955.0,974.0,52,15.0,5.0,1
...,...,...,...,...,...,...,...,...,...,...
1389,2013-08-15,Fuel (kerosene) - Retail,L,non-food,6233.0,974.0,283,15.0,15.0,8
1390,2013-09-15,Fuel (kerosene) - Retail,L,non-food,6244.0,974.0,283,15.0,15.0,8
1391,2013-10-15,Fuel (kerosene) - Retail,L,non-food,6243.0,974.0,283,15.0,15.0,8
1392,2013-11-15,Fuel (kerosene) - Retail,L,non-food,6296.0,974.0,283,15.0,15.0,8


In [67]:
food1 = food1.join(pd.get_dummies(food1['category']))
food1

,date,cmname,unit,category,price,currency,country,admname,adm1id,mktname,...,catid,sn,default,cereals and tubers,"meat, fish and eggs",milk and dairy,miscellaneous food,non-food,oil and fats,vegetables and fruits
1,2007-01-15,Rice - Retail,KG,cereals and tubers,5941.9752,IDR,Indonesia,NaN,0,National Average,...,1,974_52_15_5,1.0,1,0,0,0,0,0,0
2,2007-02-15,Rice - Retail,KG,cereals and tubers,6445.0,IDR,Indonesia,NaN,0,National Average,...,1,974_52_15_5,1.0,1,0,0,0,0,0,0
3,2007-03-15,Rice - Retail,KG,cereals and tubers,6414.0,IDR,Indonesia,NaN,0,National Average,...,1,974_52_15_5,1.0,1,0,0,0,0,0,0
4,2007-04-15,Rice - Retail,KG,cereals and tubers,6083.0,IDR,Indonesia,NaN,0,National Average,...,1,974_52_15_5,1.0,1,0,0,0,0,0,0
5,2007-05-15,Rice - Retail,KG,cereals and tubers,5955.0,IDR,Indonesia,NaN,0,National Average,...,1,974_52_15_5,1.0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,2013-08-15,Fuel (kerosene) - Retail,L,non-food,6233.0,IDR,Indonesia,NaN,0,National Average,...,8,974_283_15_15,NaN,0,0,0,0,1,0,0
1390,2013-09-15,Fuel (kerosene) - Retail,L,non-food,6244.0,IDR,Indonesia,NaN,0,National Average,...,8,974_283_15_15,NaN,0,0,0,0,1,0,0
1391,2013-10-15,Fuel (kerosene) - Retail,L,non-food,6243.0,IDR,Indonesia,NaN,0,National Average,...,8,974_283_15_15,NaN,0,0,0,0,1,0,0
1392,2013-11-15,Fuel (kerosene) - Retail,L,non-food,6296.0,IDR,Indonesia,NaN,0,National Average,...,8,974_283_15_15,NaN,0,0,0,0,1,0,0


In [68]:
food1 = food1.join(pd.get_dummies(food1['unit']))
food1

,date,cmname,unit,category,price,currency,country,admname,adm1id,mktname,...,cereals and tubers,"meat, fish and eggs",milk and dairy,miscellaneous food,non-food,oil and fats,vegetables and fruits,385 G,KG,L
1,2007-01-15,Rice - Retail,KG,cereals and tubers,5941.9752,IDR,Indonesia,NaN,0,National Average,...,1,0,0,0,0,0,0,0,1,0
2,2007-02-15,Rice - Retail,KG,cereals and tubers,6445.0,IDR,Indonesia,NaN,0,National Average,...,1,0,0,0,0,0,0,0,1,0
3,2007-03-15,Rice - Retail,KG,cereals and tubers,6414.0,IDR,Indonesia,NaN,0,National Average,...,1,0,0,0,0,0,0,0,1,0
4,2007-04-15,Rice - Retail,KG,cereals and tubers,6083.0,IDR,Indonesia,NaN,0,National Average,...,1,0,0,0,0,0,0,0,1,0
5,2007-05-15,Rice - Retail,KG,cereals and tubers,5955.0,IDR,Indonesia,NaN,0,National Average,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,2013-08-15,Fuel (kerosene) - Retail,L,non-food,6233.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,1,0,0,0,0,1
1390,2013-09-15,Fuel (kerosene) - Retail,L,non-food,6244.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,1,0,0,0,0,1
1391,2013-10-15,Fuel (kerosene) - Retail,L,non-food,6243.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,1,0,0,0,0,1
1392,2013-11-15,Fuel (kerosene) - Retail,L,non-food,6296.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,1,0,0,0,0,1


In [69]:
food1 = food1.join(pd.get_dummies(food1['date']))
food1 = food1.join(pd.get_dummies(food1['cmname']))
food1

,date,cmname,unit,category,price,currency,country,admname,adm1id,mktname,...,Chili (red) - Retail,Eggs - Retail,Fuel (kerosene) - Retail,Meat (beef) - Retail,"Meat (chicken, broiler) - Retail",Milk (condensed) - Retail,Oil (vegetable) - Retail,Rice - Retail,Sugar - Retail,Wheat - Retail
1,2007-01-15,Rice - Retail,KG,cereals and tubers,5941.9752,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,0,0,0,1,0,0
2,2007-02-15,Rice - Retail,KG,cereals and tubers,6445.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,0,0,0,1,0,0
3,2007-03-15,Rice - Retail,KG,cereals and tubers,6414.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,0,0,0,1,0,0
4,2007-04-15,Rice - Retail,KG,cereals and tubers,6083.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,0,0,0,1,0,0
5,2007-05-15,Rice - Retail,KG,cereals and tubers,5955.0,IDR,Indonesia,NaN,0,National Average,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,2013-08-15,Fuel (kerosene) - Retail,L,non-food,6233.0,IDR,Indonesia,NaN,0,National Average,...,0,0,1,0,0,0,0,0,0,0
1390,2013-09-15,Fuel (kerosene) - Retail,L,non-food,6244.0,IDR,Indonesia,NaN,0,National Average,...,0,0,1,0,0,0,0,0,0,0
1391,2013-10-15,Fuel (kerosene) - Retail,L,non-food,6243.0,IDR,Indonesia,NaN,0,National Average,...,0,0,1,0,0,0,0,0,0,0
1392,2013-11-15,Fuel (kerosene) - Retail,L,non-food,6296.0,IDR,Indonesia,NaN,0,National Average,...,0,0,1,0,0,0,0,0,0,0


In [70]:
food1 = food1.drop(columns=['category', 'unit', 'cmname', 'date' ])
food1

,price,currency,country,admname,adm1id,mktname,mktid,cmid,ptid,umid,...,Chili (red) - Retail,Eggs - Retail,Fuel (kerosene) - Retail,Meat (beef) - Retail,"Meat (chicken, broiler) - Retail",Milk (condensed) - Retail,Oil (vegetable) - Retail,Rice - Retail,Sugar - Retail,Wheat - Retail
1,5941.9752,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,...,0,0,0,0,0,0,0,1,0,0
2,6445.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,...,0,0,0,0,0,0,0,1,0,0
3,6414.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,...,0,0,0,0,0,0,0,1,0,0
4,6083.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,...,0,0,0,0,0,0,0,1,0,0
5,5955.0,IDR,Indonesia,NaN,0,National Average,974.0,52,15.0,5.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,6233.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,...,0,0,1,0,0,0,0,0,0,0
1390,6244.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,...,0,0,1,0,0,0,0,0,0,0
1391,6243.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,...,0,0,1,0,0,0,0,0,0,0
1392,6296.0,IDR,Indonesia,NaN,0,National Average,974.0,283,15.0,15.0,...,0,0,1,0,0,0,0,0,0,0


In [71]:
food1 = food1.drop(columns=['currency', 'country', 'adm1id', 'mktname', 'admname', 'sn' ])
food1

,price,mktid,cmid,ptid,umid,catid,default,cereals and tubers,"meat, fish and eggs",milk and dairy,...,Chili (red) - Retail,Eggs - Retail,Fuel (kerosene) - Retail,Meat (beef) - Retail,"Meat (chicken, broiler) - Retail",Milk (condensed) - Retail,Oil (vegetable) - Retail,Rice - Retail,Sugar - Retail,Wheat - Retail
1,5941.9752,974.0,52,15.0,5.0,1,1.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,6445.0,974.0,52,15.0,5.0,1,1.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,6414.0,974.0,52,15.0,5.0,1,1.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6083.0,974.0,52,15.0,5.0,1,1.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,5955.0,974.0,52,15.0,5.0,1,1.0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,6233.0,974.0,283,15.0,15.0,8,NaN,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1390,6244.0,974.0,283,15.0,15.0,8,NaN,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1391,6243.0,974.0,283,15.0,15.0,8,NaN,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1392,6296.0,974.0,283,15.0,15.0,8,NaN,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [72]:
food1 = food1.drop(columns=['default' ])
food1

,price,mktid,cmid,ptid,umid,catid,cereals and tubers,"meat, fish and eggs",milk and dairy,miscellaneous food,...,Chili (red) - Retail,Eggs - Retail,Fuel (kerosene) - Retail,Meat (beef) - Retail,"Meat (chicken, broiler) - Retail",Milk (condensed) - Retail,Oil (vegetable) - Retail,Rice - Retail,Sugar - Retail,Wheat - Retail
1,5941.9752,974.0,52,15.0,5.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,6445.0,974.0,52,15.0,5.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,6414.0,974.0,52,15.0,5.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,6083.0,974.0,52,15.0,5.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,5955.0,974.0,52,15.0,5.0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,6233.0,974.0,283,15.0,15.0,8,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1390,6244.0,974.0,283,15.0,15.0,8,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1391,6243.0,974.0,283,15.0,15.0,8,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1392,6296.0,974.0,283,15.0,15.0,8,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [73]:
food1.isnull().sum()

price                        0
mktid                        0
cmid                         0
ptid                         0
umid                         0
                            ..
Milk (condensed) - Retail    0
Oil (vegetable) - Retail     0
Rice - Retail                0
Sugar - Retail               0
Wheat - Retail               0
Length: 158, dtype: int64

In [85]:
# mengambil fitur/variabel masukan
X = food1.drop(columns = ['umid'])
y = food1['umid']

# membagi data menjadi data training:testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [79]:
# define kfold for cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [80]:
# Klasifikasi SVM menggunakan Cross Validation
svm = SVC()

In [86]:
cv_score_svm = cross_val_score(svm, X_train, y_train, cv=kfold, scoring="accuracy")

print('Nilai akurasi tiap fold: {}'.format(cv_score_svm))
print('Nilai akurasi rata-rata: {}'.format(cv_score_svm.mean()))

Nilai akurasi tiap fold: [0.73991031 0.71300448 0.76681614 0.76233184 0.78378378]
Nilai akurasi rata-rata: 0.7531693128105684


In [90]:
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
print(cross_validate(svm, X_train, y_train, cv=kfold, scoring=scoring))

/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1757, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/m

{'fit_time': array([0.07576823, 0.06417012, 0.05954814, 0.06089568, 0.06829739]), 'score_time': array([0.02511191, 0.02479792, 0.02464485, 0.02848244, 0.02668262]), 'test_accuracy': array([nan, nan, nan, nan, nan]), 'test_precision': array([nan, nan, nan, nan, nan]), 'test_recall': array([nan, nan, nan, nan, nan]), 'test_f1': array([nan, nan, nan, nan, nan]), 'test_roc_auc': array([nan, nan, nan, nan, nan])}


/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1757, in precision_score
    p, _, _, _ = precision_recall_fscore_support(
  File "/home/iqbalf/.local/lib/python3.8/site-packages/sklearn/m

In [88]:
params = {
        'kernel': ['linear', 'poly', 'rbf']
        }

svm_classifierCV = GridSearchCV(SVC(), params, cv= kfold, scoring= 'accuracy')
svm_classifierCV.fit(X_train,y_train)

# print parameter terbaik
print('Parameter terbaik: {0} \ndengan nilai akurasi pada data training: {1}'.format(svm_classifierCV.best_params_, svm_classifierCV.best_score_))

In [ ]:
pred = svm_classifierCV.predict(X_test)
print(classification_report(y_test, pred))